In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "12345"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('CS-340 Dashboard Malcolm Loveless'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
            dcc.Dropdown(
            id='filter-type',
            options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue/Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'unfiltered'}
            ],
            value='water'
        )
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         selected_rows=[],
                         selected_columns=[],
                         editable=False,
                         # filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="multi",
                         row_deletable=False,
                         page_action="native",
                         page_current=0,
                         page_size=5,

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
         html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if filter_type == 'water':
            # complex queries used to compile - was not sure which "Newfoundland" they wanted
            # so I added all that came up in animals.distinct('breed') that used "Newfoundland"
        dff = pd.DataFrame(db.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                              {'breed': 'Labrador Retriever Mix'},
                                                              {'breed': 'Chesa Bay Retr Mix'},
                                                              {'breed': 'Newfoundland Mix'},
                                                              {'breed': 'Newfoundland/Labrador Retriever'},
                                                              {'breed': 'Newfoundland/Australian Cattle Dog'},
                                                              {'breed': 'Newfoundland/Great Pyrenees'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                }))
    elif filter_type == 'mountain':
        dff = pd.DataFrame(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Alaskan Malamute'},
                                                              {'breed': 'Old English Sheepdog'},
                                                              {'breed': 'Rottweiler'},
                                                              {'breed': 'Siberian Husky'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 156}}]
                                                          }]
                                                }))
    
    elif filter_type == 'disaster':
        dff = pd.DataFrame(db.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                              {'breed': 'Doberman Pinscher'},
                                                              {'breed': 'German Shepherd'},
                                                              {'breed': 'Golden Retriever'},
                                                              {'breed': 'Bloodhound'},
                                                              {'breed': 'Rottweiler'}]
                                                          },
                                                          {'$and': [{'age_upon_outcome_in_weeks': {'$gte': 20}},
                                                                    {'age_upon_outcome_in_weeks': {'$lte': 300}}]
                                                          }]
                                                }))
        
    elif filter_type == 'unfiltered':
        dff = pd.DataFrame.from_records(db.read({}))
    
    data=dff.to_dict('records')    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in dff.columns]
    
              
    return (data,columns)
  

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names (breed) and values (recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500  
            )
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data"),
     Input('datatable-id', "selected_rows")])
#FIXME Add in the code for your geolocation chart
def update_map(data, selected_rows):    
    dff = pd.DataFrame.from_dict(data)
    # Default first row
    if len(selected_rows) == 0:
        # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]],
              children=[
              dl.Tooltip(dff.iloc[0,4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[0,9]),
                html.H1("Sex"),
                html.P(dff.iloc[0,12]),
                html.H1("Age"),
                html.P(dff.iloc[0,1])
                ])
              ])
           ])
                
    
        ]


    elif len(selected_rows) == 1:
        # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[0],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[0],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[0],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[0],1])
                ])
              ])
           ])
                
    
            ]
    
    elif len(selected_rows) == 2:
        # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[0],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[0],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[0],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[0],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[1],13],dff.iloc[selected_rows[1],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[1],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[1],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[1],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[1],1])
                ])
              ]) 
           ])
                
    
            ]
    elif len(selected_rows) == 3:
        # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[0],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[0],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[0],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[0],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[1],13],dff.iloc[selected_rows[1],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[1],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[1],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[1],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[1],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[2],13],dff.iloc[selected_rows[2],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[2],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[2],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[2],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[2],1])
                ])
              ])       
           ])
                
    
            ]
    elif len(selected_rows) == 3:
        # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[0],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[0],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[0],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[0],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[1],13],dff.iloc[selected_rows[1],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[1],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[1],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[1],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[1],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[2],13],dff.iloc[selected_rows[2],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[2],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[2],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[2],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[2],1])
                ])
              ])       
           ])
                
    
            ]
    elif len(selected_rows) == 4:
        # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[0],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[0],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[0],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[0],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[1],13],dff.iloc[selected_rows[1],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[1],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[1],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[1],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[1],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[2],13],dff.iloc[selected_rows[2],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[2],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[2],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[2],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[2],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[3],13],dff.iloc[selected_rows[3],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[3],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[3],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[3],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[3],1])
                ])
              ])               
           ])
                
    
            ]
    elif len(selected_rows) == 5:
        # Austin TX is at [30.75,-97.48]
        return [dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[30.75,-97.48], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[selected_rows[0],13],dff.iloc[selected_rows[0],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[0],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[0],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[0],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[0],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[1],13],dff.iloc[selected_rows[1],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[1],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[1],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[1],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[1],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[2],13],dff.iloc[selected_rows[2],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[2],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[2],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[2],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[2],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[3],13],dff.iloc[selected_rows[3],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[3],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[3],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[3],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[3],1])
                ])
              ]),
        dl.Marker(position=[dff.iloc[selected_rows[4],13],dff.iloc[selected_rows[3],14]],
              children=[
              dl.Tooltip(dff.iloc[selected_rows[4],4]),
              dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[selected_rows[4],9]),
                html.H1("Sex"),
                html.P(dff.iloc[selected_rows[4],12]),
                html.H1("Age"),
                html.P(dff.iloc[selected_rows[4],1])
                ])
              ])               
           ])
                
    
            ]

    
app.run_server(debug=True)


Dash app running on http://127.0.0.1:12427/
